In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import requests
import time
from datetime import datetime, date, time, timedelta

#for models
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression, Lasso
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from statsmodels.tsa.arima.model import ARIMA
from sklearn.neighbors import KNeighborsRegressor

#for PCA & PLS
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit, cross_val_score

#for Data
import yfinance as yf

#for Data Distribution
from scipy.stats import kurtosis, skew, shapiro

# for visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import plotly.graph_objects as go
import seaborn as sns

In [ ]:
class KNNRegressor:
    def __init__(self, k=5):
        self.k = k
        self.oKNNreg = KNeighborsRegressor(n_neighbors=k)

    def fit(self, dfX: pd.DataFrame, vY: pd.Series):
        self.oKNNreg.fit(dfX, vY) 

    def predict(self, dfX: pd.DataFrame):
        y_pred = self.oKNNreg.predict(dfX)
        return y_pred

    def score(self, dfX: pd.DataFrame, vY: pd.Series):
        R2 = self.oKNNreg.score(dfX, vY)
        return R2
    
    def train(self, splits, col_drop):
        predictions = []
        for i, (in_sample, out_of_sample) in enumerate(splits):
            X_train = in_sample.drop(columns=[col_drop])
            y_train = in_sample[col_drop]
            self.oKNNreg.fit(X_train, y_train)

            X_test = out_of_sample.drop(columns=[col_drop])
            predictions.extend(self.oKNNreg.predict(X_test))

        return predictions
    
    def evaluate(self, y_true, y_pred):
        mae = mean_absolute_error(y_true, y_pred)
        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_true, y_pred)

        # Visualize predictions vs. actual values
        plt.scatter(y_true, y_pred)
        plt.xlabel("Actual Values")
        plt.ylabel("Predicted Values")
        plt.title("Actual vs. Predicted Values")
        plt.show()

        return {
            "Mean Absolute Error": mae,
            "Mean Squared Error": mse,
            "Root Mean Squared Error": rmse,
            "R-squared": r2
        }


In [ ]:
# Example usage:
# Instantiate LinearSVRTrainer
model = KNNRegressor()

# Train the model
predictions = model.train(splits,'log_return')

# Initialize an empty list to store predictions
index_dropped = len(sp500_d)-len(predictions) ## it is always 200 from 200,20 walk forward

sp500_d_includes_results = sp500_d.copy()
sp500_d_includes_results = sp500_d_includes_results.iloc[index_dropped:, :]
sp500_d_includes_results['Predections_LinearSVM'] = predictions

# Instantiate LinearSVREvaluator
evaluation_results = model.evaluate(sp500_d_includes_results['log_return'], sp500_d_includes_results['Predections_LinearSVM'])

# Print evaluation results
for metric, value in evaluation_results.items():
    print(f"{metric}: {value}")

In [ ]:
# Assuming df is your DataFrame containing the data
# Replace 'feature1' and 'feature2' with the names of the features you want to plot
feature1 = 'log_return'
feature2 = 'Predections_LinearSVM'

# Create line plot
fig = go.Figure(data=go.Scatter(
    x=sp500_d_includes_results.index,  # Assuming the index represents x-axis values
    y=sp500_d_includes_results[feature1],
    mode='lines',
    name=feature1
))

# Add another line plot for the second feature
fig.add_trace(go.Scatter(
    x=sp500_d_includes_results.index,  # Assuming the index represents x-axis values
    y=sp500_d_includes_results[feature2],
    mode='lines',
    name=feature2
))

# Customize layout
fig.update_layout(
    title='KNN K=5 - log_return Vs. Predictions on train dataset - WF 200,20',
    xaxis=dict(title='X Axis'),  # Customize x-axis label
    yaxis=dict(title='Y Axis'),  # Customize y-axis label
    hovermode='closest'
)

# Show plot
fig.show()